<a href="https://colab.research.google.com/github/Furkanpusher/BUILDER/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 16.0 MB/s eta 0:00:00
172.5 MiB [] 0% 131.3s172.5 MiB [] 0% 24.0s172.5 MiB [] 0% 13.1s172.5 MiB [] 0% 6.0s172.5 MiB [] 1% 4.0s172.5 MiB [] 2% 3.2s172.5 MiB [] 3% 2.7s172.5 MiB [] 4% 2.5s172.5 MiB [] 5% 2.4s172.5 MiB [] 5% 2.6s172.5 MiB [] 6% 2.4s172.5 MiB [] 7% 2.3s172.5 MiB [] 8% 2.2s172.5 MiB [] 9% 2.1s172.5 MiB [] 10% 2.1s172.5 MiB [] 11% 2.0s172.5 MiB [] 12% 2.0s172.5 MiB [] 14% 1.9s172.5 MiB [] 15% 1.8s172.5 MiB [] 16% 1.7s172.5 MiB [] 17% 1.7s172.5 MiB [] 18% 1.6s172.5 MiB [] 20% 1.5s172.5 MiB [] 21% 1.5s172.5 MiB [] 23% 1.5s172.5 MiB [] 24% 1.5s172.5 MiB [] 25% 1.4s172.5 MiB [] 26% 1.5s172.5 MiB [] 28% 1.4s172.5 MiB [] 29% 1.3s172.5 MiB [] 31% 1.3s172.5 MiB [] 32% 1.2s172.5 MiB [] 33% 1.2s172.5 MiB [] 35% 1.2s172.5 MiB [] 36% 1.1s172.5 MiB [] 38% 1.1s172.5 MiB [] 39% 1.0s172.5 MiB [] 41% 1.0s172.5 MiB [] 42% 1.0s172.5 MiB [] 43% 0.9s172.5 MiB [] 45% 0.9s172.5 MiB [] 46% 0.9s172.5 MiB [] 48% 0.8s172.5 MiB [] 49% 0.8s172.5 M

In [ ]:
from playwright.async_api import async_playwright
import asyncio
import re
import json
from typing import List

async def get_all_season_fixture_links(url: str) -> dict:
    all_season_links = {}
    base_match_url = 'https://arsiv.mackolik.com/Match/Default.aspx?id='
    suffix = '#karsilastirma'

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            print(f"🚀 Ana puan durumu sayfasına bağlanıyor: {url}")
            await page.goto(url, wait_until='domcontentloaded', timeout=90000)
            await asyncio.sleep(5)

            cbo_years_selector = 'select#cboYears'
            year_options_locator = page.locator(f'{cbo_years_selector} option')
            total_years = await year_options_locator.count()

            years_to_scrape = []
            for i in range(total_years):
                year_text = await year_options_locator.nth(i).text_content()
                year_start = int(year_text.split('/')[0])

                # Sadece 1995 ve sonrası sezonları dahil et,
                # 2025-2026 sezonunu hariç tut.
                if year_start >= 1995 and not year_text.startswith('2025'):
                    year_value = await year_options_locator.nth(i).get_attribute('value')
                    years_to_scrape.append({'text': year_text, 'value': year_value})

            print(f"✅ 1995 ve sonrası ve 2025-2026 sezonu hariç toplam {len(years_to_scrape)} adet sezon bulundu.")

            for season in years_to_scrape:
                year_text = season['text']
                year_value = season['value']

                print(f"\n=======================================================")
                print(f"🔄 Sezon '{year_text}' için fikstür linkleri çekiliyor...")

                cbo_years_element = page.locator(cbo_years_selector)
                await cbo_years_element.select_option(value=year_value)

                await asyncio.sleep(5)

                fixture_tab_selector = 'h2 > a[href="#ui-tabs-2"]'
                fixture_tab_link = page.locator(fixture_tab_selector)

                if await fixture_tab_link.count() > 0:
                    await fixture_tab_link.first.click()
                    # Fikstür sekmesi açıldıktan sonra 2 saniye bekleme eklendi
                    await asyncio.sleep(2)

                    season_links = await _scrape_weeks_in_season(page, base_match_url, suffix)
                    all_season_links[year_text] = season_links
                else:
                    print(f"❌ Sezon '{year_text}' için Fikstür sekmesi bulunamadı.")

            print("\n=======================================================")
            total_links_count = sum(len(links) for links in all_season_links.values())
            print(f"✅ Tüm sezonlardan toplam {total_links_count} adet link toplandı.")

        except Exception as e:
            print(f"❌ Hata oluştu: {str(e)}")

        finally:
            await browser.close()

    return all_season_links

async def _scrape_weeks_in_season(page, base_url, suffix):
    season_links = []
    cbo_week_selector = 'select#cboWeek'

    try:
        # Görünürlük yerine sadece var olmasını bekliyoruz
        await page.wait_for_selector(f'{cbo_week_selector} option', state='attached', timeout=10000)
    except Exception as e:
        print(f"❌ Hafta seçim menüsü yüklenirken hata oluştu veya bulunamadı: {e}")
        score_links = page.locator('div#ui-tabs-2 td.score a')
        count = await score_links.count()
        if count > 0:
            print(f"     -> Bu hafta için {count} link bulundu.")
            for i in range(count):
                href_attr = await score_links.nth(i).get_attribute('href')
                if href_attr and 'popMatch(' in href_attr:
                    match_id = re.search(r'popMatch\((\d+)\)', href_attr).group(1)
                    full_link = f"{base_url}{match_id}{suffix}"
                    season_links.append(full_link)
        return season_links

    week_options = page.locator(f'{cbo_week_selector} option')
    total_weeks = await week_options.count()
    print(f"     ✅ Hafta seçim menüsü bulundu. Toplam {total_weeks} hafta mevcut.")

    for i in range(total_weeks):
        cbo_week_element = page.locator(cbo_week_selector)
        week_options_in_loop = cbo_week_element.locator('option')

        week_option_value = await week_options_in_loop.nth(i).get_attribute('value')

        if week_option_value:
            week_text = await week_options_in_loop.nth(i).text_content()
            print(f"     🔄 Hafta {i+1}/{total_weeks}: '{week_text}' maçları çekiliyor...")

            await cbo_week_element.select_option(value=week_option_value)
            await asyncio.sleep(2)

            score_links = page.locator('div#ui-tabs-2 td.score a')
            count = await score_links.count()

            for j in range(count):
                href_attr = await score_links.nth(j).get_attribute('href')
                if href_attr and 'popMatch(' in href_attr:
                    match_id = re.search(r'popMatch\((\d+)\)', href_attr).group(1)
                    full_link = f"{base_url}{match_id}{suffix}"
                    if full_link not in season_links:
                        season_links.append(full_link)

    return season_links

if __name__ == "__main__":
    standings_url = 'https://arsiv.mackolik.com/Puan-Durumu/24/INGILTERE-Premier-Lig'

    print("--- Tüm ligin 1995 sonrası sezonlar için maç linkleri toplanıyor ---")

    # Hata veren 'asyncio.run(main())' yerine doğrudan await kullanıyoruz
    all_season_data = await get_all_season_fixture_links(standings_url)

    output_filename = 'Ingiltere-Premier-Lig-All_Seasons.json'
    if all_season_data:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(all_season_data, f, indent=2, ensure_ascii=False)

        total_count = sum(len(links) for links in all_season_data.values())
        print(f"\n💾 Toplanan toplam {total_count} adet link '{output_filename}' dosyasına kaydedildi!")
    else:
        print("\n⚠️ Herhangi bir link toplanamadı.")

--- Tüm ligin 1995 sonrası sezonlar için maç linkleri toplanıyor ---
🚀 Ana puan durumu sayfasına bağlanıyor: https://arsiv.mackolik.com/Puan-Durumu/24/INGILTERE-Premier-Lig
✅ 1995 ve sonrası ve 2025-2026 sezonu hariç toplam 34 adet sezon bulundu.

🔄 Sezon '2024/2025' için fikstür linkleri çekiliyor...
     ✅ Hafta seçim menüsü bulundu. Toplam 38 hafta mevcut.
     🔄 Hafta 1/38: '1 (16.08.2024 - 19.08.2024)' maçları çekiliyor...
     🔄 Hafta 2/38: '2 (24.08.2024 - 25.08.2024)' maçları çekiliyor...
     🔄 Hafta 3/38: '3 (31.08.2024 - 1.09.2024)' maçları çekiliyor...
     🔄 Hafta 4/38: '4 (14.09.2024 - 15.09.2024)' maçları çekiliyor...
     🔄 Hafta 5/38: '5 (21.09.2024 - 22.09.2024)' maçları çekiliyor...
     🔄 Hafta 6/38: '6 (28.09.2024 - 30.09.2024)' maçları çekiliyor...
     🔄 Hafta 7/38: '7 (5.10.2024 - 6.10.2024)' maçları çekiliyor...
     🔄 Hafta 8/38: '8 (19.10.2024 - 21.10.2024)' maçları çekiliyor...
     🔄 Hafta 9/38: '9 (25.10.2024 - 27.10.2024)' maçları çekiliyor...
     🔄 Haft

/usr/local/lib/python3.11/dist-packages/playwright/_impl/_network.py:1006: RuntimeWarning: coroutine 'main' was never awaited
  self._headers_map[header["name"].lower()][header["value"]] = True


     ✅ Hafta seçim menüsü bulundu. Toplam 38 hafta mevcut.
     🔄 Hafta 1/38: '1 (10.08.2018 - 12.08.2018)' maçları çekiliyor...
     🔄 Hafta 2/38: '2 (18.08.2018 - 20.08.2018)' maçları çekiliyor...
     🔄 Hafta 3/38: '3 (25.08.2018 - 27.08.2018)' maçları çekiliyor...
     🔄 Hafta 4/38: '4 (1.09.2018 - 2.09.2018)' maçları çekiliyor...
     🔄 Hafta 5/38: '5 (15.09.2018 - 17.09.2018)' maçları çekiliyor...
     🔄 Hafta 6/38: '6 (22.09.2018 - 23.09.2018)' maçları çekiliyor...
     🔄 Hafta 7/38: '7 (29.09.2018 - 1.10.2018)' maçları çekiliyor...
     🔄 Hafta 8/38: '8 (5.10.2018 - 7.10.2018)' maçları çekiliyor...
     🔄 Hafta 9/38: '9 (20.10.2018 - 22.10.2018)' maçları çekiliyor...
     🔄 Hafta 10/38: '10 (27.10.2018 - 29.10.2018)' maçları çekiliyor...
     🔄 Hafta 11/38: '11 (3.11.2018 - 5.11.2018)' maçları çekiliyor...
     🔄 Hafta 12/38: '12 (10.11.2018 - 11.11.2018)' maçları çekiliyor...
     🔄 Hafta 13/38: '13 (24.11.2018 - 26.11.2018)' maçları çekiliyor...
     🔄 Hafta 14/38: '14 (30.11

In [ ]:
from google.colab import drive
import os
import shutil

# Google Drive'ınızı Colab'e bağlayın
drive.mount('/content/drive')

# Mevcut JSON dosyalarının bulunduğu yolları tanımlayın
# Bu yolların, dosyaların Drive'daki gerçek konumunu gösterdiğinden emin olun.
source_file_path1 = '/content/Fransa-Ligue-1-All_Seasons.json'
source_file_path2 = '/content/Türkiye-Süper-Lig-All_Seasons.json'

# Dosyaları taşımak istediğiniz yeni klasörün yolunu tanımlayın
destination_folder_path = '/content/drive/MyDrive/All_Seasons_Data'

# Hedef klasör mevcut değilse oluşturun
if not os.path.exists(destination_folder_path):
    os.makedirs(destination_folder_path)
    print(f"Klasör başarıyla oluşturuldu: {destination_folder_path}")
else:
    print(f"Klasör zaten mevcut: {destination_folder_path}")

# Dosyaları hedef klasöre taşıyın
try:
    shutil.move(source_file_path1, destination_folder_path)
    print(f"'{os.path.basename(source_file_path1)}' başarıyla taşındı.")

    shutil.move(source_file_path2, destination_folder_path)
    print(f"'{os.path.basename(source_file_path2)}' başarıyla taşındı.")

    print("\nDosyalar başarıyla yeni klasöre eklendi.")

except FileNotFoundError:
    print("\nHata: Belirtilen dosya veya klasör bulunamadı.")
except Exception as e:
    print(f"\nBir hata oluştu: {e}")

Mounted at /content/drive
Klasör başarıyla oluşturuldu: /content/drive/MyDrive/All_Seasons_Data
'Fransa-Ligue-1-All_Seasons.json' başarıyla taşındı.
'Türkiye-Süper-Lig-All_Seasons.json' başarıyla taşındı.

Dosyalar başarıyla yeni klasöre eklendi.


In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def get_match_data(url: str) -> dict:
    """
    Belirli bir maç linkinden skor, son 5 maç istatistikleri, aralarındaki maç verileri
    ve takımların maç sonucu oranlarını çeker.
    """
    match_data = {}

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            print(f"🚀 Maç sayfasına bağlanılıyor: {url}")
            await page.goto(url, wait_until='domcontentloaded', timeout=90000)

            # Sayfanın en altına kaydırılması ve verilerin yüklenmesi için bekleme
            print("⬇️ Sayfanın en altına kaydırılıyor...")
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await page.wait_for_selector('div#compare-right-coll > div:nth-child(9)', timeout=60000)
            print("✅ Gerekli tüm bloklar yüklendi.")

            # --- Skor Bilgilerini Çekme (Değişiklik yok) ---
            score_element = page.locator('div#dvScoreText')
            score_text = await score_element.text_content() if await score_element.count() > 0 else "0 - 0"
            if ' - ' in score_text:
                home_score_str, away_score_str = score_text.strip().split(' - ')
                match_data['home_team_score'] = int(home_score_str)
                match_data['away_team_score'] = int(away_score_str)
            else:
                match_data['home_team_score'] = 0
                match_data['away_team_score'] = 0

            # --- Son 5 Maç İstatistiklerini Çekme (Değişiklik yok) ---
            home_last_5_games_selector = 'div.left-block div.last-games-temp div.last-games img'
            home_games = await page.locator(home_last_5_games_selector).all()
            home_stats = {'Kazanma': 0, 'Beraberlik': 0, 'Mağlubiyet': 0}
            for game in home_games:
                image_src = await game.get_attribute('src')
                if 'G.png' in image_src:
                    home_stats['Kazanma'] += 1
                elif 'B.png' in image_src:
                    home_stats['Beraberlik'] += 1
                elif 'M.png' in image_src:
                    home_stats['Mağlubiyet'] += 1
            match_data['home_team_last_5_stats'] = home_stats

            away_last_5_games_selector = 'div.right-block div.r-last-games-temp div.last-games img'
            away_games = await page.locator(away_last_5_games_selector).all()
            away_stats = {'Kazanma': 0, 'Beraberlik': 0, 'Mağlubiyet': 0}
            for game in away_games:
                image_src = await game.get_attribute('src')
                if 'G.png' in image_src:
                    away_stats['Kazanma'] += 1
                elif 'B.png' in image_src:
                    away_stats['Beraberlik'] += 1
                elif 'M.png' in image_src:
                    away_stats['Mağlubiyet'] += 1
            match_data['away_team_last_5_stats'] = away_stats

            # --- Aralarındaki Maçlar İstatistiklerini Çekme (Değişiklik yok) ---
            print("⚔️ Aralarındaki maç istatistikleri çekiliyor...")
            other_matches_block = page.locator('div.other-matchs-temp')
            if await other_matches_block.count() > 0:
                comparison_tables = other_matches_block.locator('table.comparison-table')
                if await comparison_tables.count() >= 2:
                    all_matches_table = comparison_tables.nth(0)
                    all_matches_rows = all_matches_table.locator('tbody tr')
                    home_wins_str = await all_matches_rows.nth(1).locator('td').nth(2).text_content()
                    draws_str = await all_matches_rows.nth(2).locator('td').nth(2).text_content()
                    away_wins_str = await all_matches_rows.nth(3).locator('td').nth(2).text_content()
                    match_data['all_matches'] = {
                        'home_wins': int(home_wins_str),
                        'draws': int(draws_str),
                        'away_wins': int(away_wins_str)
                    }
                    home_matches_table = comparison_tables.nth(1)
                    home_matches_rows = home_matches_table.locator('tbody tr')
                    home_wins_home_str = await home_matches_rows.nth(1).locator('td').nth(2).text_content()
                    draws_home_str = await home_matches_rows.nth(2).locator('td').nth(2).text_content()
                    away_wins_home_str = await home_matches_rows.nth(3).locator('td').nth(2).text_content()
                    match_data['home_matches'] = {
                        'home_wins': int(home_wins_home_str),
                        'draws': int(draws_home_str),
                        'away_wins': int(away_wins_home_str)
                    }
                else:
                    print("⚠️ Aralarındaki maçlar bölümünde yeterli tablo bulunamadı.")
                    match_data['all_matches'] = None
                    match_data['home_matches'] = None
            else:
                print("⚠️ 'Aralarındaki Maçlar' bölümü bulunamadı.")
                match_data['all_matches'] = None
                match_data['home_matches'] = None

            # --- Oran Verilerini Çekme (nth-child Selector'ları Kullanımı) ---
            print("💰 Bahis oranları çekiliyor...")
            try:
                # Ev sahibi takımın oranlarını çekme
                home_odds_block = page.locator('#compare-right-coll > div:nth-child(9)')
                if await home_odds_block.count() > 0:
                    home_odds_table = home_odds_block.locator('table.md-table3').first
                    home_win_odds = await home_odds_table.locator('tbody tr').nth(0).locator('td').nth(1).text_content()
                    home_draw_odds = await home_odds_table.locator('tbody tr').nth(1).locator('td').nth(1).text_content()
                    match_data['home_team_odds'] = {
                        'win_odds': float(home_win_odds.strip()),
                        'draw_odds': float(home_draw_odds.strip())
                    }
                else:
                    print("⚠️ Ev sahibi oran tablosu bulunamadı.")
                    match_data['home_team_odds'] = None

                # Deplasman takımının oranlarını çekme
                away_odds_block = page.locator('#compare-right-coll > div:nth-child(10)')
                if await away_odds_block.count() > 0:
                    away_odds_table = away_odds_block.locator('table.md-table3').first
                    away_win_odds = await away_odds_table.locator('tbody tr').nth(0).locator('td').nth(1).text_content()
                    away_draw_odds = await away_odds_table.locator('tbody tr').nth(1).locator('td').nth(1).text_content()
                    match_data['away_team_odds'] = {
                        'win_odds': float(away_win_odds.strip()),
                        'draw_odds': float(away_draw_odds.strip())
                    }
                else:
                    print("⚠️ Deplasman oran tablosu bulunamadı.")
                    match_data['away_team_odds'] = None

            except Exception as e:
                print(f"❌ Oran verileri çekilirken hata oluştu: {str(e)}")
                match_data['home_team_odds'] = None
                match_data['away_team_odds'] = None

            print(f"✅ Veriler başarıyla çekildi.")

        except Exception as e:
            print(f"❌ Genel bir hata oluştu: {str(e)}")
            match_data = None

        finally:
            await browser.close()

    return match_data

async def main():
    test_url = 'https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma'
    data = await get_match_data(test_url)

    if data:
        print("\n=====================================")
        print("Çekilen Maç Verileri:")
        print(f"Ev Sahibi Takım Golü: {data.get('home_team_score')}")
        print(f"Deplasman Takımı Golü: {data.get('away_team_score')}")
        print("-------------------------------------")
        print("Ev Sahibi Takım (Son 5 Maç):")
        print(f"  Kazanma: {data.get('home_team_last_5_stats', {}).get('Kazanma')}")
        print(f"  Beraberlik: {data.get('home_team_last_5_stats', {}).get('Beraberlik')}")
        print(f"  Mağlubiyet: {data.get('home_team_last_5_stats', {}).get('Mağlubiyet')}")
        print("-------------------------------------")
        print("Deplasman Takımı (Son 5 Maç):")
        print(f"  Kazanma: {data.get('away_team_last_5_stats', {}).get('Kazanma')}")
        print(f"  Beraberlik: {data.get('away_team_last_5_stats', {}).get('Beraberlik')}")
        print(f"  Mağlubiyet: {data.get('away_team_last_5_stats', {}).get('Mağlubiyet')}")
        print("-------------------------------------")
        print("Aralarındaki Tüm Maçlar:")
        if data.get('all_matches'):
            print(f"  Ev Sahibi Galibiyet: {data['all_matches']['home_wins']}")
            print(f"  Beraberlik: {data['all_matches']['draws']}")
            print(f"  Deplasman Galibiyet: {data['all_matches']['away_wins']}")
        else:
            print("  Veri çekilemedi.")
        print("-------------------------------------")
        print("Ev Sahibi Takım Evindeki Maçlar:")
        if data.get('home_matches'):
            print(f"  Ev Sahibi Galibiyet: {data['home_matches']['home_wins']}")
            print(f"  Beraberlik: {data['home_matches']['draws']}")
            print(f"  Deplasman Galibiyet: {data['home_matches']['away_wins']}")
        else:
            print("  Veri çekilemedi.")
        print("-------------------------------------")
        print("MAÇ SONUCU ORANLARI:")
        if data.get('home_team_odds'):
            print(f"  Ev Sahibi Galibiyet Oranı: {data['home_team_odds']['win_odds']}")
            print(f"  Ev Sahibi Beraberlik Oranı: {data['home_team_odds']['draw_odds']}")
        if data.get('away_team_odds'):
            print(f"  Deplasman Galibiyet Oranı: {data['away_team_odds']['win_odds']}")
            print(f"  Deplasman Beraberlik Oranı: {data['away_team_odds']['draw_odds']}")
        if not data.get('home_team_odds') and not data.get('away_team_odds'):
            print("  Oran verileri çekilemedi.")
        print("=====================================")
    else:
        print("Veri çekilemedi.")

if __name__ == '__main__':
    asyncio.run(main())

🚀 Maç sayfasına bağlanılıyor: https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma
⬇️ Sayfanın en altına kaydırılıyor...
❌ Genel bir hata oluştu: Page.wait_for_selector: Timeout 60000ms exceeded.
Call log:
  - waiting for locator("div.md-table3") to be visible

Veri çekilemedi.


In [ ]:
!pip install nest_asyncio

ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("select#cboYears option").nth(95)\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("select#cboYears option").nth(95)



In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright, TimeoutError
import re

nest_asyncio.apply()

async def get_match_odds_efficiently(url: str) -> dict:
    """
    Belirli bir maç linkinden verileri çeker ve işler.
    Takım toplam değeri için farklı bir URL'ye navigasyon yapar.
    """
    all_odds = {}

    # Ev sahibi takımı için maç sonucu selector'ları
    home_team_selectors = {
        'home_galibiyet_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_beraberlik_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_kaybetme_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için handikap selector'ları
    home_handicap_selectors = {
        'home_handicap_win': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_handicap_draw': '#compare-right-coll > div:nth-child(12) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_handicap_lose': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için ilk yarı oranları selector'ları
    home_first_half_selectors = {
        'home_ilk_yari_kazanma': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_ilk_yari_berabere': '#compare-right-coll > div:nth-child(15) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_ilk_yari_kaybetme': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # "Karşılıklı Gol Var/Yok" selector'ları
    kg_selectors = {
        'kg_var': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'kg_yok': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "1.5 Gol Alt/Üst" selector'ları
    over_under_1_5_selectors = {
        'home_1.5_ust': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_1.5_alt': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "2.5 Gol Alt/Üst" selector'ları
    over_under_2_5_selectors = {
        'home_2.5_ust': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_2.5_alt': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "3.5 Gol Alt/Üst" selector'ları
    over_under_3_5_selectors = {
        'home_3.5_ust': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_3.5_alt': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            print(f"🚀 Maç sayfasına bağlanılıyor: {url}")
            await page.goto(url, wait_until='domcontentloaded', timeout=90000)

            # Oranları çekme işlemleri
            print("\n-------------------------------------")
            print("⏳ Maç oranları çekiliyor...")

            selectors_to_process = {
                'home_team_selectors': home_team_selectors,
                'home_handicap_selectors': home_handicap_selectors,
                'home_first_half_selectors': home_first_half_selectors,
                'kg_selectors': kg_selectors,
                'over_under_1_5_selectors': over_under_1_5_selectors,
                'over_under_2_5_selectors': over_under_2_5_selectors,
                'over_under_3_5_selectors': over_under_3_5_selectors,
            }

            for group_name, selector_group in selectors_to_process.items():
                print(f"  - {group_name} çekiliyor...")
                for key, selector in selector_group.items():
                    try:
                        locator = page.locator(selector)
                        content = await locator.text_content(timeout=30000)
                        all_odds[key] = float(content.strip()) if content else None
                    except TimeoutError:
                        all_odds[key] = None
                    except Exception:
                        all_odds[key] = None
                print(f"  ✅ {group_name} tamamlandı.")

            # Sakat oyuncu sayılarını ve toplam değerlerini çekme
            print("\n-------------------------------------")
            print("⏳ Sakat oyuncu sayıları ve piyasa değerleri hesaplanıyor...")

            home_injured_total_worth = 0.0
            away_injured_total_worth = 0.0
            home_injured_count = 0
            away_injured_count = 0

            injured_sections_locator = page.locator('.detail-title:has-text("Sakat/Cezalılar")')
            injured_sections_count = await injured_sections_locator.count()

            if injured_sections_count > 0:
                home_section_locator = injured_sections_locator.nth(0).locator('xpath=..')
                home_injured_players_locator = home_section_locator.locator('table tr:has(td:text("Sakat"))')
                home_injured_count = await home_injured_players_locator.count()

                for i in range(home_injured_count):
                    player_row = home_injured_players_locator.nth(i)
                    worth_text = await player_row.locator('td:nth-child(4)').text_content()
                    if worth_text:
                        cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                        try:
                            home_injured_total_worth += float(cleaned_worth)
                        except ValueError:
                            pass

                if injured_sections_count > 1:
                    away_section_locator = injured_sections_locator.nth(1).locator('xpath=..')
                    away_injured_players_locator = away_section_locator.locator('table tr:has(td:text("Sakat"))')
                    away_injured_count = await away_injured_players_locator.count()

                    for i in range(away_injured_count):
                        player_row = away_injured_players_locator.nth(i)
                        worth_text = await player_row.locator('td:nth-child(4)').text_content()
                        if worth_text:
                            cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                            try:
                                away_injured_total_worth += float(cleaned_worth)
                            except ValueError:
                                pass

            all_odds['home_injured_count'] = home_injured_count
            all_odds['away_injured_count'] = away_injured_count
            all_odds['home_injured_total_worth'] = round(home_injured_total_worth, 2)
            all_odds['away_injured_total_worth'] = round(away_injured_total_worth, 2)
            all_odds['total_injured_count'] = home_injured_count + away_injured_count

            print(f"✅ Ev sahibi sakat oyuncu sayısı: {all_odds['home_injured_count']}")
            print(f"✅ Ev sahibi sakatların toplam değeri (M €): {all_odds['home_injured_total_worth']}")
            print(f"✅ Deplasman sakat oyuncu sayısı: {all_odds['away_injured_count']}")
            print(f"✅ Deplasman sakatların toplam değeri (M €): {all_odds['away_injured_total_worth']}")
            print(f"✅ Toplam sakat oyuncu sayısı: {all_odds['total_injured_count']}")

            print("-------------------------------------")

            # Takım toplam değerlerini ayrı bir sayfadan çekme
            print("\n-------------------------------------")
            print("⏳ Takım toplam değerleri ayrı bir sayfadan çekiliyor...")

            worth_url = url.replace("#karsilastirma", "#mac-bilgisi")
            await page.goto(worth_url, wait_until='domcontentloaded', timeout=90000)

            def clean_and_convert_worth(worth_text):
                if not worth_text:
                    return None

                # İstenmeyen tüm metinleri ve boşlukları temizle
                cleaned_worth = worth_text.replace('İlk 11 Değeri :', '').replace('EUR', '').replace('M €', '').strip()

                # Binlik ayraçlarını kaldır (noktaları) ve ondalık ayıracını (virgülü) noktaya çevir
                cleaned_worth = cleaned_worth.replace('.', '').replace(',', '.')

                try:
                    return float(cleaned_worth)
                except ValueError:
                    return None

            try:
                # Ev sahibi takım değeri
                home_worth_selector = '#dvHomeSquad > table > tbody > tr.team-value > td'
                home_worth_text = await page.locator(home_worth_selector).text_content(timeout=30000)
                all_odds['home_total_worth'] = clean_and_convert_worth(home_worth_text)
            except (TimeoutError, ValueError) as e:
                print(f"❌ Ev sahibi takım değeri çekilemedi: {e}")
                all_odds['home_total_worth'] = None

            try:
                # Deplasman takım değeri
                away_worth_selector = '#dvAwaySquad > table > tbody > tr.team-value-2 > td'
                away_worth_text = await page.locator(away_worth_selector).text_content(timeout=30000)
                all_odds['away_total_worth'] = clean_and_convert_worth(away_worth_text)
            except (TimeoutError, ValueError) as e:
                print(f"❌ Deplasman takım değeri çekilemedi: {e}")
                all_odds['away_total_worth'] = None

            print(f"✅ Ev sahibi takım toplam değeri: {all_odds['home_total_worth']}")
            print(f"✅ Deplasman takım toplam değeri: {all_odds['away_total_worth']}")
            print("-------------------------------------")

            # Deplasman takım oranlarını ev sahibi oranlarına göre atama
            print("\n-------------------------------------")
            print("🔄 Deplasman takım oranları atılıyor...")
            all_odds['away_galibiyet_orani'] = all_odds.get('home_kaybetme_orani')
            all_odds['away_beraberlik_orani'] = all_odds.get('home_beraberlik_orani')
            all_odds['away_kaybetme_orani'] = all_odds.get('home_galibiyet_orani')

            all_odds['away_handicap_win'] = all_odds.get('home_handicap_lose')
            all_odds['away_handicap_draw'] = all_odds.get('home_handicap_draw')
            all_odds['away_handicap_lose'] = all_odds.get('home_handicap_win')

            all_odds['away_ilk_yari_kazanma'] = all_odds.get('home_ilk_yari_kaybetme')
            all_odds['away_ilk_yari_berabere'] = all_odds.get('home_ilk_yari_berabere')
            all_odds['away_ilk_yari_kaybetme'] = all_odds.get('home_ilk_yari_kazanma')

            print("✅ Deplasman takım oranları başarıyla atandı.")
            print("-------------------------------------")

            # Maç sonucu hesaplama
            print("\n-------------------------------------")
            print("⏳ Maç sonucu hesaplanıyor...")

            all_odds['HomeWin'] = 0
            all_odds['HomeDraw'] = 0
            all_odds['HomeLose'] = 0

            try:
                score_text = await page.locator('#dvScoreText').text_content(timeout=10000)
                scores = score_text.strip().split(' - ')
                if len(scores) == 2:
                    home_score = int(scores[0])
                    away_score = int(scores[1])

                    if home_score > away_score:
                        all_odds['HomeWin'] = 1
                    elif home_score == away_score:
                        all_odds['HomeDraw'] = 1
                    else:
                        all_odds['HomeLose'] = 1

                    print(f"✅ Maç sonucu başarıyla hesaplandı. Skor: {home_score} - {away_score}")
                    print(f"   - HomeWin: {all_odds['HomeWin']}")
                    print(f"   - HomeDraw: {all_odds['HomeDraw']}")
                    print(f"   - HomeLose: {all_odds['HomeLose']}")

            except Exception as e:
                print(f"❌ Maç sonucu çekilemedi veya hesaplanamadı: {e}")
                all_odds['HomeWin'] = None
                all_odds['HomeDraw'] = None
                all_odds['HomeLose'] = None

            print("-------------------------------------")


        except Exception as e:
            print(f"❌ Sayfaya bağlanırken genel hata oluştu: {str(e)}")
        finally:
            await browser.close()

    return all_odds

async def main():
    test_url = 'https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma'
    match_odds_data = await get_match_odds_efficiently(test_url)

    if match_odds_data:
        print("\n=====================================")
        print("Çekilen Tüm Veriler:")
        for key, value in match_odds_data.items():
            print(f"  {key}: {value}")
        print("=====================================")
    else:
        print("Hiçbir veri çekilemedi.")

if __name__ == '__main__':
    asyncio.run(main())

🚀 Maç sayfasına bağlanılıyor: https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma

-------------------------------------
⏳ Maç oranları çekiliyor...
  - home_team_selectors çekiliyor...
  ✅ home_team_selectors tamamlandı.
  - home_handicap_selectors çekiliyor...
  ✅ home_handicap_selectors tamamlandı.
  - home_first_half_selectors çekiliyor...
  ✅ home_first_half_selectors tamamlandı.
  - kg_selectors çekiliyor...
  ✅ kg_selectors tamamlandı.
  - over_under_1_5_selectors çekiliyor...
  ✅ over_under_1_5_selectors tamamlandı.
  - over_under_2_5_selectors çekiliyor...
  ✅ over_under_2_5_selectors tamamlandı.
  - over_under_3_5_selectors çekiliyor...
  ✅ over_under_3_5_selectors tamamlandı.

-------------------------------------
⏳ Sakat oyuncu sayıları ve piyasa değerleri hesaplanıyor...
✅ Ev sahibi sakat oyuncu sayısı: 3
✅ Ev sahibi sakatların toplam değeri (M €): 0.55
✅ Deplasman sakat oyuncu sayısı: 5
✅ Deplasman sakatların toplam değeri (M €): 0.8
✅ Toplam sakat oyun

# DİREKT CSV'YE CEVIRME

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright, TimeoutError
import re
import csv

nest_asyncio.apply()

async def get_match_odds_efficiently(url: str) -> dict:
    """
    Belirli bir maç linkinden verileri çeker ve işler.
    Takım toplam değeri için farklı bir URL'ye navigasyon yapar.
    """
    all_odds = {}

    # Ev sahibi takımı için maç sonucu selector'ları
    home_team_selectors = {
        'home_galibiyet_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_beraberlik_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_kaybetme_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için handikap selector'ları
    home_handicap_selectors = {
        'home_handicap_win': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_handicap_draw': '#compare-right-coll > div:nth-child(12) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_handicap_lose': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için ilk yarı oranları selector'ları
    home_first_half_selectors = {
        'home_ilk_yari_kazanma': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_ilk_yari_berabere': '#compare-right-coll > div:nth-child(15) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_ilk_yari_kaybetme': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # "Karşılıklı Gol Var/Yok" selector'ları
    kg_selectors = {
        'kg_var': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'kg_yok': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "1.5 Gol Alt/Üst" selector'ları
    over_under_1_5_selectors = {
        'home_1.5_ust': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_1.5_alt': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "2.5 Gol Alt/Üst" selector'ları
    over_under_2_5_selectors = {
        'home_2.5_ust': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_2.5_alt': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "3.5 Gol Alt/Üst" selector'ları
    over_under_3_5_selectors = {
        'home_3.5_ust': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_3.5_alt': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            print(f"🚀 Maç sayfasına bağlanılıyor: {url}")
            await page.goto(url, wait_until='domcontentloaded', timeout=90000)

            # Oranları çekme işlemleri
            print("\n-------------------------------------")
            print("⏳ Maç oranları çekiliyor...")

            selectors_to_process = {
                'home_team_selectors': home_team_selectors,
                'home_handicap_selectors': home_handicap_selectors,
                'home_first_half_selectors': home_first_half_selectors,
                'kg_selectors': kg_selectors,
                'over_under_1_5_selectors': over_under_1_5_selectors,
                'over_under_2_5_selectors': over_under_2_5_selectors,
                'over_under_3_5_selectors': over_under_3_5_selectors,
            }

            for group_name, selector_group in selectors_to_process.items():
                print(f"  - {group_name} çekiliyor...")
                for key, selector in selector_group.items():
                    try:
                        locator = page.locator(selector)
                        content = await locator.text_content(timeout=30000)
                        all_odds[key] = float(content.strip()) if content else None
                    except TimeoutError:
                        all_odds[key] = None
                    except Exception:
                        all_odds[key] = None
                print(f"  ✅ {group_name} tamamlandı.")

            # Sakat oyuncu sayılarını ve toplam değerlerini çekme
            print("\n-------------------------------------")
            print("⏳ Sakat oyuncu sayıları ve piyasa değerleri hesaplanıyor...")

            home_injured_total_worth = 0.0
            away_injured_total_worth = 0.0
            home_injured_count = 0
            away_injured_count = 0

            injured_sections_locator = page.locator('.detail-title:has-text("Sakat/Cezalılar")')
            injured_sections_count = await injured_sections_locator.count()

            if injured_sections_count > 0:
                home_section_locator = injured_sections_locator.nth(0).locator('xpath=..')
                home_injured_players_locator = home_section_locator.locator('table tr:has(td:text("Sakat"))')
                home_injured_count = await home_injured_players_locator.count()

                for i in range(home_injured_count):
                    player_row = home_injured_players_locator.nth(i)
                    worth_text = await player_row.locator('td:nth-child(4)').text_content()
                    if worth_text:
                        cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                        try:
                            home_injured_total_worth += float(cleaned_worth)
                        except ValueError:
                            pass

                if injured_sections_count > 1:
                    away_section_locator = injured_sections_locator.nth(1).locator('xpath=..')
                    away_injured_players_locator = away_section_locator.locator('table tr:has(td:text("Sakat"))')
                    away_injured_count = await away_injured_players_locator.count()

                    for i in range(away_injured_count):
                        player_row = away_injured_players_locator.nth(i)
                        worth_text = await player_row.locator('td:nth-child(4)').text_content()
                        if worth_text:
                            cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                            try:
                                away_injured_total_worth += float(cleaned_worth)
                            except ValueError:
                                pass

            all_odds['home_injured_count'] = home_injured_count
            all_odds['away_injured_count'] = away_injured_count
            all_odds['home_injured_total_worth'] = round(home_injured_total_worth, 2)
            all_odds['away_injured_total_worth'] = round(away_injured_total_worth, 2)
            all_odds['total_injured_count'] = home_injured_count + away_injured_count

            print(f"✅ Ev sahibi sakat oyuncu sayısı: {all_odds['home_injured_count']}")
            print(f"✅ Ev sahibi sakatların toplam değeri (M €): {all_odds['home_injured_total_worth']}")
            print(f"✅ Deplasman sakat oyuncu sayısı: {all_odds['away_injured_count']}")
            print(f"✅ Deplasman sakatların toplam değeri (M €): {all_odds['away_injured_total_worth']}")
            print(f"✅ Toplam sakat oyuncu sayısı: {all_odds['total_injured_count']}")

            print("-------------------------------------")

            # Takım toplam değerlerini ayrı bir sayfadan çekme
            print("\n-------------------------------------")
            print("⏳ Takım toplam değerleri ayrı bir sayfadan çekiliyor...")

            worth_url = url.replace("#karsilastirma", "#mac-bilgisi")
            await page.goto(worth_url, wait_until='domcontentloaded', timeout=90000)

            def clean_and_convert_worth(worth_text):
                if not worth_text:
                    return None

                # İstenmeyen tüm metinleri ve boşlukları temizle
                cleaned_worth = worth_text.replace('İlk 11 Değeri :', '').replace('EUR', '').replace('M €', '').strip()

                # Binlik ayraçlarını kaldır (noktaları) ve ondalık ayıracını (virgülü) noktaya çevir
                cleaned_worth = cleaned_worth.replace('.', '').replace(',', '.')

                try:
                    return float(cleaned_worth)
                except ValueError:
                    return None

            try:
                # Ev sahibi takım değeri
                home_worth_selector = '#dvHomeSquad > table > tbody > tr.team-value > td'
                home_worth_text = await page.locator(home_worth_selector).text_content(timeout=30000)
                all_odds['home_total_worth'] = clean_and_convert_worth(home_worth_text)
            except (TimeoutError, ValueError) as e:
                print(f"❌ Ev sahibi takım değeri çekilemedi: {e}")
                all_odds['home_total_worth'] = None

            try:
                # Deplasman takım değeri
                away_worth_selector = '#dvAwaySquad > table > tbody > tr.team-value-2 > td'
                away_worth_text = await page.locator(away_worth_selector).text_content(timeout=30000)
                all_odds['away_total_worth'] = clean_and_convert_worth(away_worth_text)
            except (TimeoutError, ValueError) as e:
                print(f"❌ Deplasman takım değeri çekilemedi: {e}")
                all_odds['away_total_worth'] = None

            print(f"✅ Ev sahibi takım toplam değeri: {all_odds['home_total_worth']}")
            print(f"✅ Deplasman takım toplam değeri: {all_odds['away_total_worth']}")
            print("-------------------------------------")

            # Deplasman takım oranlarını ev sahibi oranlarına göre atama
            print("\n-------------------------------------")
            print("🔄 Deplasman takım oranları atılıyor...")
            all_odds['away_galibiyet_orani'] = all_odds.get('home_kaybetme_orani')
            all_odds['away_beraberlik_orani'] = all_odds.get('home_beraberlik_orani')
            all_odds['away_kaybetme_orani'] = all_odds.get('home_galibiyet_orani')

            all_odds['away_handicap_win'] = all_odds.get('home_handicap_lose')
            all_odds['away_handicap_draw'] = all_odds.get('home_handicap_draw')
            all_odds['away_handicap_lose'] = all_odds.get('home_handicap_win')

            all_odds['away_ilk_yari_kazanma'] = all_odds.get('home_ilk_yari_kaybetme')
            all_odds['away_ilk_yari_berabere'] = all_odds.get('home_ilk_yari_berabere')
            all_odds['away_ilk_yari_kaybetme'] = all_odds.get('home_ilk_yari_kazanma')

            print("✅ Deplasman takım oranları başarıyla atandı.")
            print("-------------------------------------")

            # Maç sonucu hesaplama
            print("\n-------------------------------------")
            print("⏳ Maç sonucu hesaplanıyor...")

            all_odds['HomeWin'] = 0
            all_odds['HomeDraw'] = 0
            all_odds['HomeLose'] = 0

            try:
                score_text = await page.locator('#dvScoreText').text_content(timeout=10000)
                scores = score_text.strip().split(' - ')
                if len(scores) == 2:
                    home_score = int(scores[0])
                    away_score = int(scores[1])

                    if home_score > away_score:
                        all_odds['HomeWin'] = 1
                    elif home_score == away_score:
                        all_odds['HomeDraw'] = 1
                    else:
                        all_odds['HomeLose'] = 1

                    print(f"✅ Maç sonucu başarıyla hesaplandı. Skor: {home_score} - {away_score}")
                    print(f"   - HomeWin: {all_odds['HomeWin']}")
                    print(f"   - HomeDraw: {all_odds['HomeDraw']}")
                    print(f"   - HomeLose: {all_odds['HomeLose']}")

            except Exception as e:
                print(f"❌ Maç sonucu çekilemedi veya hesaplanamadı: {e}")
                all_odds['HomeWin'] = None
                all_odds['HomeDraw'] = None
                all_odds['HomeLose'] = None

            print("-------------------------------------")

        except Exception as e:
            print(f"❌ Sayfaya bağlanırken genel hata oluştu: {str(e)}")
        finally:
            await browser.close()

    return all_odds


async def save_to_csv(data: dict, filename: str):
    """
    Sözlük verisini bir CSV dosyasına yazar.
    """
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = data.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerow(data)
    print(f"✅ Veri '{filename}' dosyasına başarıyla kaydedildi.")


async def main():
    test_url = 'https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma'
    match_odds_data = await get_match_odds_efficiently(test_url)

    if match_odds_data:
        print("\n=====================================")
        print("Çekilen Tüm Veriler:")
        for key, value in match_odds_data.items():
            print(f"  {key}: {value}")
        print("=====================================")

        # Veriyi CSV'ye kaydetme
        await save_to_csv(match_odds_data, 'mac_verileri.csv')
    else:
        print("Hiçbir veri çekilemedi.")

if __name__ == '__main__':
    asyncio.run(main())

🚀 Maç sayfasına bağlanılıyor: https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma

-------------------------------------
⏳ Maç oranları çekiliyor...
  - home_team_selectors çekiliyor...
  ✅ home_team_selectors tamamlandı.
  - home_handicap_selectors çekiliyor...
  ✅ home_handicap_selectors tamamlandı.
  - home_first_half_selectors çekiliyor...
  ✅ home_first_half_selectors tamamlandı.
  - kg_selectors çekiliyor...
  ✅ kg_selectors tamamlandı.
  - over_under_1_5_selectors çekiliyor...
  ✅ over_under_1_5_selectors tamamlandı.
  - over_under_2_5_selectors çekiliyor...
  ✅ over_under_2_5_selectors tamamlandı.
  - over_under_3_5_selectors çekiliyor...
  ✅ over_under_3_5_selectors tamamlandı.

-------------------------------------
⏳ Sakat oyuncu sayıları ve piyasa değerleri hesaplanıyor...
✅ Ev sahibi sakat oyuncu sayısı: 3
✅ Ev sahibi sakatların toplam değeri (M €): 0.55
✅ Deplasman sakat oyuncu sayısı: 5
✅ Deplasman sakatların toplam değeri (M €): 0.8
✅ Toplam sakat oyun

# .JSON --> .CSV


In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright, TimeoutError
import re
import json

nest_asyncio.apply()

async def get_match_odds_efficiently(url: str) -> dict:
    """
    Belirli bir maç linkinden verileri çeker ve işler.
    Takım toplam değeri için farklı bir URL'ye navigasyon yapar.
    """
    all_odds = {}

    # Ev sahibi takımı için maç sonucu selector'ları
    home_team_selectors = {
        'home_galibiyet_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_beraberlik_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_kaybetme_orani': '#compare-right-coll > div:nth-child(9) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için handikap selector'ları
    home_handicap_selectors = {
        'home_handicap_win': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_handicap_draw': '#compare-right-coll > div:nth-child(12) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_handicap_lose': '#compare-right-coll > div:nth-child(12) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # Ev sahibi takımı için ilk yarı oranları selector'ları
    home_first_half_selectors = {
        'home_ilk_yari_kazanma': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(2) > td:nth-child(2) > span',
        'home_ilk_yari_berabere': '#compare-right-coll > div:nth-child(15) > table > tbody > tr.row.alt2 > td:nth-child(2) > span',
        'home_ilk_yari_kaybetme': '#compare-right-coll > div:nth-child(15) > table > tbody > tr:nth-child(4) > td:nth-child(2) > span'
    }

    # "Karşılıklı Gol Var/Yok" selector'ları
    kg_selectors = {
        'kg_var': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'kg_yok': '#compare-right-coll > div:nth-child(18) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "1.5 Gol Alt/Üst" selector'ları
    over_under_1_5_selectors = {
        'home_1.5_ust': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_1.5_alt': '#compare-right-coll > div:nth-child(20) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "2.5 Gol Alt/Üst" selector'ları
    over_under_2_5_selectors = {
        'home_2.5_ust': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_2.5_alt': '#compare-right-coll > div:nth-child(23) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    # "3.5 Gol Alt/Üst" selector'ları
    over_under_3_5_selectors = {
        'home_3.5_ust': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt1 > td:nth-child(2) > span',
        'home_3.5_alt': '#compare-right-coll > div:nth-child(26) > table > tbody > tr.row.alt2 > td:nth-child(2) > span'
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url, wait_until='domcontentloaded', timeout=90000)

            # Oranları çekme işlemleri
            selectors_to_process = {
                'home_team_selectors': home_team_selectors,
                'home_handicap_selectors': home_handicap_selectors,
                'home_first_half_selectors': home_first_half_selectors,
                'kg_selectors': kg_selectors,
                'over_under_1_5_selectors': over_under_1_5_selectors,
                'over_under_2_5_selectors': over_under_2_5_selectors,
                'over_under_3_5_selectors': over_under_3_5_selectors,
            }

            for group_name, selector_group in selectors_to_process.items():
                for key, selector in selector_group.items():
                    try:
                        locator = page.locator(selector)
                        content = await locator.text_content(timeout=30000)
                        all_odds[key] = float(content.strip()) if content else None
                    except (TimeoutError, Exception):
                        all_odds[key] = None

            # Sakat oyuncu sayılarını ve toplam değerlerini çekme
            home_injured_total_worth = 0.0
            away_injured_total_worth = 0.0
            home_injured_count = 0
            away_injured_count = 0

            injured_sections_locator = page.locator('.detail-title:has-text("Sakat/Cezalılar")')
            injured_sections_count = await injured_sections_locator.count()

            if injured_sections_count > 0:
                home_section_locator = injured_sections_locator.nth(0).locator('xpath=..')
                home_injured_players_locator = home_section_locator.locator('table tr:has(td:text("Sakat"))')
                home_injured_count = await home_injured_players_locator.count()

                for i in range(home_injured_count):
                    player_row = home_injured_players_locator.nth(i)
                    worth_text = await player_row.locator('td:nth-child(4)').text_content()
                    if worth_text:
                        cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                        try:
                            home_injured_total_worth += float(cleaned_worth)
                        except ValueError:
                            pass

                if injured_sections_count > 1:
                    away_section_locator = injured_sections_locator.nth(1).locator('xpath=..')
                    away_injured_players_locator = away_section_locator.locator('table tr:has(td:text("Sakat"))')
                    away_injured_count = await away_injured_players_locator.count()

                    for i in range(away_injured_count):
                        player_row = away_injured_players_locator.nth(i)
                        worth_text = await player_row.locator('td:nth-child(4)').text_content()
                        if worth_text:
                            cleaned_worth = worth_text.strip().replace(' M €', '').replace('.', '').replace(',', '.')
                            try:
                                away_injured_total_worth += float(cleaned_worth)
                            except ValueError:
                                pass

            all_odds['home_injured_count'] = home_injured_count
            all_odds['away_injured_count'] = away_injured_count
            all_odds['home_injured_total_worth'] = round(home_injured_total_worth, 2)
            all_odds['away_injured_total_worth'] = round(away_injured_total_worth, 2)
            all_odds['total_injured_count'] = home_injured_count + away_injured_count

            # Takım toplam değerlerini ayrı bir sayfadan çekme
            worth_url = url.replace("#karsilastirma", "#mac-bilgisi")
            await page.goto(worth_url, wait_until='domcontentloaded', timeout=90000)

            def clean_and_convert_worth(worth_text):
                if not worth_text:
                    return None
                cleaned_worth = worth_text.replace('İlk 11 Değeri :', '').replace('EUR', '').replace('M €', '').strip()
                cleaned_worth = cleaned_worth.replace('.', '').replace(',', '.')
                try:
                    return float(cleaned_worth)
                except ValueError:
                    return None

            try:
                home_worth_selector = '#dvHomeSquad > table > tbody > tr.team-value > td'
                home_worth_text = await page.locator(home_worth_selector).text_content(timeout=30000)
                all_odds['home_total_worth'] = clean_and_convert_worth(home_worth_text)
            except (TimeoutError, ValueError):
                all_odds['home_total_worth'] = None

            try:
                away_worth_selector = '#dvAwaySquad > table > tbody > tr.team-value-2 > td'
                away_worth_text = await page.locator(away_worth_selector).text_content(timeout=30000)
                all_odds['away_total_worth'] = clean_and_convert_worth(away_worth_text)
            except (TimeoutError, ValueError):
                all_odds['away_total_worth'] = None

            # Deplasman takım oranlarını ev sahibi oranlarına göre atama
            all_odds['away_galibiyet_orani'] = all_odds.get('home_kaybetme_orani')
            all_odds['away_beraberlik_orani'] = all_odds.get('home_beraberlik_orani')
            all_odds['away_kaybetme_orani'] = all_odds.get('home_galibiyet_orani')
            all_odds['away_handicap_win'] = all_odds.get('home_handicap_lose')
            all_odds['away_handicap_draw'] = all_odds.get('home_handicap_draw')
            all_odds['away_handicap_lose'] = all_odds.get('home_handicap_win')
            all_odds['away_ilk_yari_kazanma'] = all_odds.get('home_ilk_yari_kaybetme')
            all_odds['away_ilk_yari_berabere'] = all_odds.get('home_ilk_yari_berabere')
            all_odds['away_ilk_yari_kaybetme'] = all_odds.get('home_ilk_yari_kazanma')

            # Maç sonucu hesaplama
            all_odds['HomeWin'] = 0
            all_odds['HomeDraw'] = 0
            all_odds['HomeLose'] = 0

            try:
                score_text = await page.locator('#dvScoreText').text_content(timeout=10000)
                scores = score_text.strip().split(' - ')
                if len(scores) == 2:
                    home_score = int(scores[0])
                    away_score = int(scores[1])

                    if home_score > away_score:
                        all_odds['HomeWin'] = 1
                    elif home_score == away_score:
                        all_odds['HomeDraw'] = 1
                    else:
                        all_odds['HomeLose'] = 1
            except Exception:
                all_odds['HomeWin'] = None
                all_odds['HomeDraw'] = None
                all_odds['HomeLose'] = None

        except Exception as e:
            print(f"❌ Sayfaya bağlanırken genel hata oluştu: {str(e)}")
            return None
        finally:
            await browser.close()

    return all_odds


async def save_all_data_to_json(data_list: list, filename: str):
    """
    Tüm maç verilerini içeren listeyi tek bir JSON dosyasına yazar.
    """
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(data_list, jsonfile, ensure_ascii=False, indent=4)
    print(f"✅ Tüm veriler '{filename}' dosyasına başarıyla kaydedildi.")


async def main():
    # Örnek linkler
    match_urls = [
        'https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma',
        # ... diğer 298 link buraya eklenecek ...
    ]

    all_match_data = []

    print("🚀 Veri çekme işlemi başlıyor...")
    for i, url in enumerate(match_urls):
        print(f"\n-------------------------------------")
        print(f"⏳ Link {i+1}/{len(match_urls)} işleniyor: {url}")

        match_data = await get_match_odds_efficiently(url)

        if match_data:
            all_match_data.append(match_data)
            print(f"✅ Link {i+1} başarıyla çekildi.")
        else:
            print(f"❌ Link {i+1} çekilirken hata oluştu.")

        print(f"-------------------------------------")

    if all_match_data:
        # Tüm veriyi tek bir JSON dosyasına kaydet
        await save_all_data_to_json(all_match_data, 'tum_mac_verileri.json')
    else:
        print("Hiçbir veri çekilemedi.")

if __name__ == '__main__':
    asyncio.run(main())

🚀 Veri çekme işlemi başlıyor...

-------------------------------------
⏳ Link 1/1 işleniyor: https://arsiv.mackolik.com/Match/Default.aspx?id=2843645#karsilastirma
✅ Link 1 başarıyla çekildi.
-------------------------------------
✅ Tüm veriler 'tum_mac_verileri.json' dosyasına başarıyla kaydedildi.
